# Identify "cloudy transition" Spitzer exoplanet host targets 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.utils.data import download_file
from astropy.table import Table, join
import astropy.units as u

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

In [3]:
cold_url = 'https://irsa.ipac.caltech.edu/data/SPITZER/docs/files/spitzer/extrasolarplanets.txt'
warm_url = 'http://ssc.spitzer.caltech.edu/warmmission/scheduling/observinglogs/extrasolarplanetsWarm.txt'

In [31]:
start_line = 24
cold_table = Table.read(cold_url, format='ascii.fixed_width', 
                        header_start=start_line, data_start=start_line + 3, delimiter=' ')

In [32]:
start_line = 17
warm_table = Table.read(warm_url, format='ascii.fixed_width', 
                        header_start=start_line, data_start=start_line + 3, delimiter=' ')

In [76]:
all_channels = ['ch1', 'ch2', 'ch3', 'ch4']

observations = dict()

for table in [cold_table, warm_table]: 
    for target in table['TargetName']: 
        if '_' in target: 
            str1, str2, *rest = target.split('_')
        elif '-' in target: 
            str1, str2, *rest = target.split('-')
        else: 
            str1, str2, rest = target, "", ""
            
        if str1 in all_channels: 
            channel = str1
            name = str2

        elif str2 in all_channels: 
            channel = str2
            name = str1

        # if no string is channel, make simple planet name: 
        if str1 not in all_channels and str2 not in all_channels: 
            name = str1 + str2 + ' '.join(rest)

        fixed_name = (name.upper().replace(' ', '').replace('ECL', '')
                      .replace('EC', '').replace('CH24', '').replace('CH13', '')
                      .replace('-', '').replace('DAR', ''))
        
        if fixed_name.endswith('B') or fixed_name.endswith('S') or fixed_name.endswith('C'): 
            fixed_name = fixed_name[:-1]
            
        if fixed_name.endswith('CH') or fixed_name.endswith('CP'): 
            fixed_name = fixed_name[:-2]
        
        fixed_name = fixed_name.lower()
        
        if fixed_name in observations: 
            observations[fixed_name].add(channel)
        else: 
            observations[fixed_name] = {channel}

In [121]:
obs_table = Table(names=['NAME_LOWERCASE', "CH1", "CH2"], dtype=['S832', bool, bool])
for obs in observations: 
    row = [obs, 'ch1' in observations[obs], 'ch2' in observations[obs]]
    obs_table.add_row(row)
obs_table = Table(obs_table)

In [134]:
archive = NasaExoplanetArchive.get_confirmed_planets_table(cache=False)

In [135]:
archive['NAME_LOWERCASE'] = [target[:-1].replace('-', '') for target in archive['NAME_LOWERCASE']]

In [138]:
observed_planets = join(archive, obs_table, join_type='inner', keys=['NAME_LOWERCASE'])

In [150]:
teff = observed_planets['st_teff']  # stellar effective temperature
rstar = observed_planets['st_rad']  # stellar radius
a = observed_planets['pl_orbsmax']  # orbital semimajor axis

teq = (teff * np.sqrt(rstar / (2 * a))).decompose()

//anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:466: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
//anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:466: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [152]:
cloudy_transition = (1300 * u.K < teq) & (1700 * u.K > teq)

print('\n'.join(observed_planets[cloudy_transition]['pl_hostname'].data.data))

51 Peg
CoRoT-2
HAT-P-1
HAT-P-16
HAT-P-2
HATS-3
HD 219828
HD 75289
Kepler-7
Kepler-9
Qatar-1
Qatar-2
tau Boo
TrES-3
WASP-104
WASP-127
WASP-131
WASP-140
WASP-2
WASP-21
WASP-28
WASP-31
WASP-37
WASP-4
WASP-43
WASP-46
WASP-50
WASP-64
